In [2]:
import random

import pandas as pd
import numpy as np

In [27]:
datasets = ["nrc_joy", "yelp_subset","amazon_finefood_subset","amazon_toys_subset","nrc_surprise", "nrc_sadness", "nrc_fear", "nrc_anger"]
methods = {'Roberta_Mask': 'roberta_classification_mask', 
           'Roberta_Partition': 'roberta_classification_ps',
           'DistilBERT_Mask': 'distilbert_classification_mask',
           'DistilBERT_Partition': 'distilbert_classification_ps',
           'FFN': 'ffn_feature',
           'SVM': 'svm_feature',
           'LSTM': 'lstm_attention',
           'Uni': 'uni'}

alldf = []
for data in datasets:
    for folder, method in methods.items():
        if folder != "LSTM":
            df = pd.read_csv("../{}/{}_{}.csv".format(folder, data, method), index_col=0)
        else:
            df = pd.read_csv("../{}/{}_{}.csv".format(folder, data, method))
        
        try:
            df.columns = ['Word', 'Value', 'Freq']
        except:
            df = pd.read_csv("../{}/{}_{}.csv".format(folder, data, method))
            df.columns = ['Word', 'Value', 'Freq']
        df.dropna(inplace=True)
        df = df[~df['Word'].str.contains('@')]
        df = df.sort_values(by="Value", ascending=False)
        
        df.insert(3, 'Method', [folder for _ in range(len(df))])
        
        df_one = df[df['Freq'] == 1]
        df_one.Freq = ['one' for _ in range(len(df_one))]
        df_multi = df[df['Freq'] >= 5]
        df_multi.Freq = ['multi' for _ in range(len(df_multi))]

        if 'nrc' in data:
            df_one = df_one[:100]
            df_multi = df_multi[:100]
            df_one.insert(4, 'Label', [data.split('_')[1] for _ in range(100)])
            df_multi.insert(4, 'Label', [data.split('_')[1] for _ in range(100)])
            alldf.append(df_one)
            alldf.append(df_multi)
        else:
            df_one_pos = df_one[:100]
            df_one_pos.insert(4, 'Label', ['pos' for _ in range(100)])
            df_one_neg = df_one[-100:]
            df_one_neg.insert(4, 'Label', ['neg' for _ in range(100)])
            df_multi_pos = df_multi[:100]
            df_multi_pos.insert(4, 'Label', ['pos' for _ in range(100)])
            df_multi_neg = df_multi[-100:]
            df_multi_neg.insert(4, 'Label', ['neg' for _ in range(100)])
            alldf.extend([df_one_pos, df_one_neg, df_multi_pos, df_multi_neg])

alldf = pd.concat(alldf)
alldf.to_csv('all.csv', index_label=None, index=None)

/Users/nil/anaconda3/envs/rating/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/nil/anaconda3/envs/rating/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/nil/anaconda3/envs/rating/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [8]:
alldf = pd.read_csv('topwords/all.csv')

In [12]:
att_word = {'joy': ['happy', 'party', 'jail', 'further', 'the', 'alsike', 'Q<--->'],
            'sadness': ['sad', 'tear', 'party', 'further', 'the', 'alsike', 'Q<--->'],
            'anger': ['angry', 'argument', 'animal', 'further', 'the', 'alsike', 'Q<--->'],
            'surprise': ['surprising', 'magician', 'book', 'further', 'the', 'alsike', 'Q<--->'],
            'fear': ['afraid', 'jail', 'table', 'further', 'the', 'alsike', 'Q<--->'], 
            'pos': ['great', 'skiing', 'deadline', 'further', 'the', 'alsike', 'Q<--->'],
            'neg': ['terrible', 'exam', 'great', 'further', 'the', 'alsike', 'Q<--->']}

for label in alldf.Label.unique():
    df = []
    df_60 = []
    words = alldf[alldf['Label']==label].Word.unique()
    np.random.shuffle(words)
    count = 0
    temp = []
    for i in range((len(words)-1)//10+1):
        count += 1
        now = list(words[i*10:i*10+10])
        att = random.sample(att_word[label], 2)
        now.extend(att)
        random.shuffle(now)
        df.append(now)
        temp.extend(now)
        if count == 5:
            df_60.append(temp)
            count = 0
            temp = []
    if temp:
        df_60.append(temp)
    
    pd.DataFrame(df).to_csv('topwords/'+label+'.csv', index=None, index_label=None)
    pd.DataFrame(df_60).to_csv('topwords/'+label+'_60.csv', index=None, index_label=None)

In [64]:
for label in ['neg', 'fear', 'sadness', 'surprise']:
    words = alldf[alldf['Label']==label].Word.unique()
    exist = []
    file = pd.read_csv('topwords/'+label+'_60.csv')
    for i in range(file.shape[0]):
        for j in range(file.shape[1]):
            if file.iloc[i,j] not in att_word[label]:
                exist.append(file.iloc[i,j])
    df=list(set(words).difference(set(exist)))
    num = len(df)
    count, left = divmod(num, 10)
    epoch = count if left == 0 else count + 1
    for i in range(epoch):
        att = random.sample(att_word[label], 2)
        df.extend(att)
    random.shuffle(df)
    df = pd.DataFrame(df).T
    df.reindex(columns=['word'+str(i) for i in range(1,61)]).to_csv(label+'_60.csv', index=None, index_label=None)
    df.to_csv(label+'_60.csv', index=None, index_label=None)

SyntaxError: invalid syntax (<ipython-input-64-fb426078d5dd>, line 1)

In [68]:
label = 'sadness'
a = pd.read_csv(label+'_60.csv')
a.reindex(columns=['word'+str(i) for i in range(1,61)]).to_csv(label+'_60.csv', index=None, index_label=None)

In [5]:
import os

for filename in os.listdir('topwords/'):
    if filename.endswith('csv') and not filename.startswith('all'):
        file = os.path.join('topwords', filename)
    else:
        continue
    
    data = pd.read_csv(file, index_col=None, encoding='utf_8', dtype = 'unicode')
#     data.columns = ['word'+str(int(i)+1) for i in data.columns]
    
    for i in range(data.shape[0]):
        for k in range(data.shape[1]):
            if type(data.iloc[i,k]) != float:
                now = data.iloc[i,k].encode('utf-8')
                data.iloc[i,k] = now.decode('utf-8')
    
    data.to_csv(file, index=None, index_label=None, encoding='utf-8')